### 準備資料 ###   
**格式**
```json
{
    'movie01':{
        'user01':4.0,
        'user02':3.0,
        'user03':4.0,
        'user05':2.0
    },
    'movie02':{
        'user02':3.0,
        'user03':3.0,
        'user07':2.0,
        'user05':4.0
    }
}
```

In [1]:
import os
with open(os.getcwd()+'/ml-1m/users.dat','r') as f:
    f_users = f.readlines()
    
with open(os.getcwd()+'/ml-1m/movies.dat','r') as f:
    f_movies = f.readlines()
    
with open(os.getcwd()+'/ml-1m/ratings.dat','r') as f:
    f_ratings = f.readlines();

In [2]:
#prepare user dictionary
users = {}
for line in f_users:
    (uid,sex,age,occupation,zipcode) = line.split('::')
    if uid not in users:
        users[uid] = {}
    users[uid]['gender'] = sex,
    users[uid]['age'] = int(age)
    
#prepare movie name dictionary
movies = {}
for line in f_movies:
    (mid,title,genres) = line.split('::')
    movies[mid] = title

#define a rating dictionary
critics = {}
for line in f_ratings:
    (uid,mid,rating,timestamp) = line.split('::')

    if mid not in critics:
        critics[mid] = {}
    critics[mid][uid] = float(rating)
#     movie_name = movies[mid]
#     if movie_name not in critics:
#         critics[movie_name] = {}
#     critics[movie_name][uid] = float(rating)

### 定義函式 ###   
**相似度**   
原本的 function: pearson   
嘗試修改的 function: pearsonCustom
   
**分群**   
原本的 function: hcluster   
嘗試修改的 function: hclusterCustom   

In [3]:
from math import sqrt

#origin function
def pearson(v1, v2):
  # Simple sums
    sum1 = sum(v1)
    sum2 = sum(v2)

  # Sums of the squares
    sum1Sq = sum([pow(v, 2) for v in v1])
    sum2Sq = sum([pow(v, 2) for v in v2])

  # Sum of the products
    pSum = sum([v1[i] * v2[i] for i in range(len(v1))])

  # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / len(v1)
    den = sqrt((sum1Sq - pow(sum1, 2) / len(v1)) * (sum2Sq - pow(sum2, 2)
               / len(v1)))
    if den == 0:
        return 0

    return 1.0 - num / den

#new function by jo
def pearsonCustom(v1, v2, threshold=5):
    # Get the list of mutually rated items
    si = {}
    for item in v1:
        if item in v2:
            si[item] = 1
    # If they are no ratings in common, return 0
    if len(si) == 0:
        return 1
    if len(si) < threshold:
        return 1
    # Sum calculations
    n = len(si)
    # Sums of all the preferences
    sum1 = sum([v1[it] for it in si])
    sum2 = sum([v2[it] for it in si])
    # Sums of the squares
    sum1Sq = sum([pow(v1[it], 2) for it in si])
    sum2Sq = sum([pow(v2[it], 2) for it in si])
    # Sum of the products
    pSum = sum([v1[it] * v2[it] for it in si])
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    r = 1.0 - num / den
    return r
    

In [4]:
class bicluster:

    def __init__(
        self,
        vec,
        left=None,
        right=None,
        distance=0.0,
        id=None,
        ):
        self.left = left
        self.right = right
        self.vec = vec
        self.id = id
        self.distance = distance


#original function
def hcluster(rows, distance=pearson):
    distances = {}
    currentclustid = -1

  # Clusters are initially just the rows
    clust = [bicluster(rows[i], id=i) for i in range(len(rows))]
    
    while len(clust) > 1:
        lowestpair = (0, 1)
        closest = distance(clust[0].vec, clust[1].vec)

    # loop through every pair looking for the smallest distance
        for i in range(len(clust)):
            for j in range(i + 1, len(clust)):
        # distances is the cache of distance calculations
                if (clust[i].id, clust[j].id) not in distances:
                    distances[(clust[i].id, clust[j].id)] = \
                        distance(clust[i].vec, clust[j].vec)

                d = distances[(clust[i].id, clust[j].id)]

                if d < closest:
                    closest = d
                    lowestpair = (i, j)

    # calculate the average of the two clusters
        mergevec = [(clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i])
                    / 2.0 for i in range(len(clust[0].vec))]

    # create the new cluster
        newcluster = bicluster(mergevec, left=clust[lowestpair[0]],
                               right=clust[lowestpair[1]], distance=closest,
                               id=currentclustid)

    # cluster ids that weren't in the original set are negative
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)

    return clust[0]

#new function by jo
def hclusterCustom(prefs, threshold=5, distance=pearsonCustom):
    distances = {}
    currentclustid = -1

  # Clusters are initially just the rows
    clust = [bicluster(prefs[r], id=r) for r in prefs]
    
    while len(clust) > 1:
        lowestpair = (0, 1)
        closest = distance(clust[0].vec, clust[1].vec, threshold)

    # loop through every pair looking for the smallest distance
        for i in range(len(clust)):
            for j in range(i + 1, len(clust)):
        # distances is the cache of distance calculations
                if (clust[i].id, clust[j].id) not in distances:
                    distances[(clust[i].id, clust[j].id)] = \
                        distance(clust[i].vec, clust[j].vec, threshold)

                d = distances[(clust[i].id, clust[j].id)]

                if d < closest:
                    closest = d
                    lowestpair = (i, j)

    # calculate the average of the two clusters
        mergevec = {}
        
        for u in clust[lowestpair[0]].vec:
            if u in clust[lowestpair[1]].vec:
                score = (clust[lowestpair[0]].vec[u] + clust[lowestpair[1]].vec[u])/2.0
                mergevec[u] = score
            else:
                mergevec[u] = clust[lowestpair[0]].vec[u]
        for u in clust[lowestpair[1]].vec:
            if u in mergevec:
                continue
            mergevec[u] = clust[lowestpair[1]].vec[u]
#         mergevec = [(clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i])
#                     / 2.0 for i in range(len(clust[0].vec))]

    # create the new cluster
        newcluster = bicluster(mergevec, left=clust[lowestpair[0]],
                               right=clust[lowestpair[1]], distance=closest,
                               id=currentclustid)

    # cluster ids that weren't in the original set are negative
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)

    return clust[0]

In [5]:
import time
start = time.time()

cluster_result = hclusterCustom(critics, threshold=10)

print "spent:" , time.time()-start , " seconds"

spent: 8425.32519794  seconds


In [6]:
def printcluster(clust, labels=None, n=0):
  # indent to make a hierarchy layout
    for i in range(n):
        print ' ',
    if clust.id < 0:
    # negative id means that this is branch
        print '-'
    else:
    # positive id means that this is an endpoint
        if labels == None:
            print clust.id
        else:
            print labels[clust.id]

  # now print the right and left branches
    if clust.left != None:
        printcluster(clust.left, labels=labels, n=n + 1)
    if clust.right != None:
        printcluster(clust.right, labels=labels, n=n + 1)
        
    

In [7]:
printcluster(cluster_result,labels=movies)

-
  -
    -
      -
        Love Walked In (1998)
        Strike! (a.k.a. All I Wanna Do, The Hairy Bird) (1998)
      -
        Baby, The (1973)
        Mighty Peking Man (Hsing hsing wang) (1977)
    -
      -
        Theodore Rex (1995)
        Tashunga (1995)
      -
        Show, The (1995)
        Six Ways to Sunday (1997)
  -
    -
      -
        -
          -
            -
              -
                -
                  Acid House, The (1998)
                  Slappy and the Stinkers (1998)
                -
                  Mascara (1999)
                  Resurrection Man (1998)
              -
                -
                  Dingo (1992)
                  Blood and Sand (Sangre y Arena) (1989)
                -
                  Hippie Revolution, The (1996)
                  Little Indian, Big City (Un indien dans la ville) (1994)
            -
              -
                -
                  -
                    -
                      -
                     

In [8]:
def getheight(clust):
  # Is this an endpoint? Then the height is just 1
    if clust.left == None and clust.right == None:
        return 1

  # Otherwise the height is the same of the heights of
  # each branch
    return getheight(clust.left) + getheight(clust.right)


def getdepth(clust):
  # The distance of an endpoint is 0.0
    if clust.left == None and clust.right == None:
        return 0

  # The distance of a branch is the greater of its two sides
  # plus its own distance
    return max(getdepth(clust.left), getdepth(clust.right)) + clust.distance


def drawdendrogram(clust, labels, jpeg='clusters.jpg', ext='JPEG'):
  # height and width
    h = getheight(clust) * 20
    w = 1500
    depth = getdepth(clust)

  # width is fixed, so scale distances accordingly
    scaling = float(w - 150) / depth

  # Create a new image with a white background
    img = Image.new('RGB', (w, h), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    draw.line((0, h / 2, 10, h / 2), fill=(255, 0, 0))

  # Draw the first node
    drawnode(
        draw,
        clust,
        10,
        h / 2,
        scaling,
        labels,
        )
    img.save(jpeg, ext)


def drawnode(
    draw,
    clust,
    x,
    y,
    scaling,
    labels,
    ):
    if clust.id < 0:
        h1 = getheight(clust.left) * 20
        h2 = getheight(clust.right) * 20
        top = y - (h1 + h2) / 2
        bottom = y + (h1 + h2) / 2
    # Line length
        ll = clust.distance * scaling
    # Vertical line from this cluster to children
        draw.line((x, top + h1 / 2, x, bottom - h2 / 2), fill=(255, 0, 0))

    # Horizontal line to left item
        draw.line((x, top + h1 / 2, x + ll, top + h1 / 2), fill=(255, 0, 0))

    # Horizontal line to right item
        draw.line((x, bottom - h2 / 2, x + ll, bottom - h2 / 2), fill=(255, 0,
                  0))

    # Call the function to draw the left and right nodes
        drawnode(
            draw,
            clust.left,
            x + ll,
            top + h1 / 2,
            scaling,
            labels,
            )
        drawnode(
            draw,
            clust.right,
            x + ll,
            bottom - h2 / 2,
            scaling,
            labels,
            )
    else:
    # If this is an endpoint, draw the item label
        draw.text((x + 5, y - 7), labels[clust.id], (0, 0, 0))

In [9]:
from PIL import Image, ImageDraw
drawdendrogram(cluster_result,movies,jpeg='movies_overlap10.png',ext='png')


In [ ]:
# 執行的話 browser 會當掉
# from IPython.display import Image
# Image(filename='movies.png')

some code for debugging
---

In [ ]:
movie1 = 'Toy Story (1995)'
movie2 = 'Toy Story 2 (1999)'
pearsonCustom(critics, movie1, movie2)

In [101]:
minValue = 100
maxValue = 1
count = 1
for r in critics:
    if len(critics[r]) < minValue:
        minValue = len(critics[r])
        minmovie = r
    if len(critics[r]) > maxValue:
        maxValue = len(critics[r])
        maxmovie = r

print 'min=', minValue, ', minmovie=', movies[minmovie], ',', minmovie
print 'max=', maxValue, ', maxmovie=', movies[maxmovie], ',', maxmovie
print critics.items()[0]

min= 1 , minmovie= Project Moon Base (1953) , 3779
max= 3428 , maxmovie= American Beauty (1999) , 2858
('1718', {'3934': 2.0, '4821': 3.0, '3841': 3.0, '1470': 1.0, '5059': 4.0, '4768': 3.0, '2820': 2.0})


In [73]:
critics[minmovie]

{'3338': 3.0}